In [1]:
import os

import pandas as pd
import numpy as np

from shared_astro_utils.matching_utils import match_galaxies_to_catalog_pandas

from zoobot.shared import label_metadata, schemas

In [2]:
# final version of https://github.com/mwalmsley/curation-datasets/blob/main/notebooks/rename_gz2_catalog.ipynb

In [3]:
# hart = pd.read_csv('/run/media/walml/beta/galaxy_zoo/gz2/subjects/gz2_hart16.csv')
# len(hart)

In [4]:
# TODO I think is is a renaming of hart 2016
master_df = pd.read_parquet('/run/media/walml/beta/galaxy_zoo/gz2/subjects/image_master_catalog.parquet')
"""
original    209294
extra        21844
stripe82      8557
"""
len(master_df)


239695

In [5]:
master_df = master_df.query('sample == "original"')
len(master_df)

209294

In [6]:
renamer = {
    't01_smooth_or_features_a01_smooth': 'smooth-or-featured-gz2_smooth',
    't01_smooth_or_features_a02_features_or_disk': 'smooth-or-featured-gz2_featured-or-disk',
    't01_smooth_or_features_a03_star_or_artifact': 'smooth-or-featured-gz2_artifact',
    't02_edgeon_a04_yes': 'disk-edge-on-gz2_yes',
    't02_edgeon_a05_no': 'disk-edge-on-gz2_no',
    't03_bar_a06_bar': 'bar-gz2_yes',
    't03_bar_a07_no_bar': 'bar-gz2_no',
    't04_spiral_a08_spiral': 'has-spiral-arms-gz2_yes',
    't04_spiral_a09_no_spiral': 'has-spiral-arms-gz2_no',
    't05_bulge_prominence_a10_no_bulge': 'bulge-size-gz2_no',
    't05_bulge_prominence_a11_just_noticeable': 'bulge-size-gz2_just-noticeable',
    't05_bulge_prominence_a12_obvious': 'bulge-size-gz2_obvious',
    't05_bulge_prominence_a13_dominant': 'bulge-size-gz2_dominant',
    't06_odd_a14_yes': 'something-odd-gz2_yes',
    't06_odd_a15_no': 'something-odd-gz2_no',
    't07_rounded_a16_completely_round': 'how-rounded-gz2_round',
    't07_rounded_a17_in_between': 'how-rounded-gz2_in-between',
    't07_rounded_a18_cigar_shaped': 'how-rounded-gz2_cigar',
    't09_bulge_shape_a25_rounded': 'bulge-shape-gz2_round',
    't09_bulge_shape_a26_boxy': 'bulge-shape-gz2_boxy',
    't09_bulge_shape_a27_no_bulge': 'bulge-shape-gz2_no-bulge',
    't10_arms_winding_a28_tight': 'spiral-winding-gz2_tight',
    't10_arms_winding_a29_medium': 'spiral-winding-gz2_medium',
    't10_arms_winding_a30_loose': 'spiral-winding-gz2_loose',
    't11_arms_number_a31_1': 'spiral-arm-count-gz2_1',
    't11_arms_number_a32_2': 'spiral-arm-count-gz2_2',
    't11_arms_number_a33_3': 'spiral-arm-count-gz2_3',
    't11_arms_number_a34_4': 'spiral-arm-count-gz2_4',
    't11_arms_number_a36_more_than_4': 'spiral-arm-count-gz2_more-than-4',
    't11_arms_number_a37_cant_tell': 'spiral-arm-count-gz2_cant-tell'
}

renamer_with_count = dict([(key + '_count', value) for key, value in renamer.items()])

# copy the key id columns
df = master_df[['dr7objid', 'png_loc', 'ra', 'dec']]

# copy all label columns using renamer
for key, value in renamer_with_count.items():
    df.loc[:, value] = master_df[key]

df


/tmp/ipykernel_24973/3437701375.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, value] = master_df[key]
/tmp/ipykernel_24973/3437701375.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, value] = master_df[key]
/tmp/ipykernel_24973/3437701375.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,dr7objid,png_loc,ra,dec,smooth-or-featured-gz2_smooth,smooth-or-featured-gz2_featured-or-disk,smooth-or-featured-gz2_artifact,disk-edge-on-gz2_yes,disk-edge-on-gz2_no,bar-gz2_yes,...,bulge-shape-gz2_no-bulge,spiral-winding-gz2_tight,spiral-winding-gz2_medium,spiral-winding-gz2_loose,spiral-arm-count-gz2_1,spiral-arm-count-gz2_2,spiral-arm-count-gz2_3,spiral-arm-count-gz2_4,spiral-arm-count-gz2_more-than-4,spiral-arm-count-gz2_cant-tell
0,587732591714893851,/raid/scratch/walml/galaxy_zoo/gz2/png/587732/...,179.042984,60.522518,0,42,3,2,40,1,...,0,27,12,1,0,3,1,2,18,16
1,588009368545984617,/raid/scratch/walml/galaxy_zoo/gz2/png/588009/...,135.084396,52.494240,1,41,0,0,41,4,...,0,35,4,2,0,4,2,1,21,13
2,587732484359913515,/raid/scratch/walml/galaxy_zoo/gz2/png/587732/...,183.371979,50.741508,28,5,3,0,5,2,...,0,0,0,0,0,0,0,0,0,0
3,587741723357282317,/raid/scratch/walml/galaxy_zoo/gz2/png/587741/...,186.251953,28.558598,1,27,0,1,26,1,...,1,16,8,1,0,2,6,5,6,6
4,587738410866966577,/raid/scratch/walml/galaxy_zoo/gz2/png/587738/...,161.086395,14.084465,33,8,2,1,7,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239683,587739608626888793,/raid/scratch/walml/galaxy_zoo/gz2/png/587739/...,183.491501,32.117870,35,12,3,12,0,0,...,10,0,0,0,0,0,0,0,0,0
239684,587742572147376208,/raid/scratch/walml/galaxy_zoo/gz2/png/587742/...,167.420471,18.718920,33,13,3,0,13,5,...,0,2,0,1,0,2,0,0,0,1
239686,587729158440681699,/raid/scratch/walml/galaxy_zoo/gz2/png/587729/...,220.151138,3.538694,33,8,1,0,8,0,...,0,0,0,0,0,0,0,0,0,0
239687,587734841742459081,/raid/scratch/walml/galaxy_zoo/gz2/png/587734/...,118.103683,23.507954,32,0,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:

# now the schema object will work, I can calculate fractions and totals

question_answer_pairs = label_metadata.gz2_ortho_pairs
dependencies = label_metadata.gz2_ortho_dependencies

# dependencies = {
#     'smooth-or-featured': None,  # always asked
#     'disk-edge-on': 'smooth-or-featured_featured-or-disk',
#     'has-spiral-arms': 'smooth-or-featured_featured-or-disk',
#     'bar': 'smooth-or-featured_featured-or-disk',
#     'bulge-size': 'smooth-or-featured_featured-or-disk',
#     'how-rounded': 'smooth-or-featured_smooth',
#     'bulge-shape': 'disk-edge-on_yes',  # gz2 only
#     'edge-on-bulge': 'disk-edge-on_yes',
#     'spiral-winding': 'has-spiral-arms_yes',
#     'spiral-arm-count': 'has-spiral-arms_yes',
#     'something-odd': None  # actually neglects the artifact branch
# }

schema = schemas.Schema(question_answer_pairs, dependencies)
for question in schema.questions:
    df[question.text + '_total-votes'] = df[[a.text for a in question.answers]].sum(axis=1)

    for answer in question.answers:
        df[answer.text + '_fraction'] = df[answer.text] / df[question.text + '_total-votes']





/tmp/ipykernel_24973/1510733872.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[question.text + '_total-votes'] = df[[a.text for a in question.answers]].sum(axis=1)
/tmp/ipykernel_24973/1510733872.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[answer.text + '_fraction'] = df[answer.text] / df[question.text + '_total-votes']
/tmp/ipykernel_24973/1510733872.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [8]:
# fix paths
# will eventually remove this prepended string
df['png_loc'] = df['png_loc'].str.replace('/raid/scratch/walml', '/run/media/walml/beta')
df['png_loc'][0]


/tmp/ipykernel_24973/1056972041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['png_loc'] = df['png_loc'].str.replace('/raid/scratch/walml', '/run/media/walml/beta')


'/run/media/walml/beta/galaxy_zoo/gz2/png/587732/587732591714893851.png'

In [9]:
# takes about 2 mins on lofar9
# df['png_exists'] = df['png_loc'].apply(os.path.isfile)

In [10]:
# df[~df['png_exists']].squeeze()['png_loc']
# # '/raid/scratch/walml/galaxy_zoo/gz2/png/587736/587736915146703138.png'
# # literally 1 image missing...

In [11]:


df = df[df['dr7objid'] != 587736915146703138]

# label_df['file_exists'].value_counts()

In [20]:

def copy_jpg(png_loc):
    jpg_loc = png_loc.replace('png', 'jpg').replace('/run/media/walml/beta/galaxy_zoo/gz2' , '/home/walml/repos/galaxy-datasets/roots/gz2')
    jpg_dir = os.path.dirname(jpg_loc)
    # if not os.path.isdir(jpg_dir):
    #     os.mkdir(jpg_dir)
    if not os.path.isfile(jpg_loc):
        print(f'Making {jpg_loc}')
        Image.open(png_loc).save(jpg_loc)


In [21]:
# df['png_loc'].str.replace('png', 'jpg').str.replace('/run/media/walml/beta/galaxy_zoo/gz2' , '/home/walml/repos/galaxy-datasets/roots/gz2').iloc[0]

In [23]:
# now make jpg version of every png
# takes about 40 mins from scratch

from PIL import Image

from multiprocessing import Pool

pool = Pool(20)

_ = [_ for x in pool.imap_unordered(copy_jpg, df['png_loc'])]


In [24]:
df['file_loc'] = df['png_loc'].str.replace('png', 'jpg').str.replace('/run/media/walml/beta/galaxy_zoo/gz2' , '/home/walml/repos/galaxy-datasets/roots/gz2')
df['subfolder'] = df['file_loc'].apply(lambda x: os.path.basename(os.path.dirname(x)))

/tmp/ipykernel_24973/3819007290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['file_loc'] = df['png_loc'].str.replace('png', 'jpg').str.replace('/run/media/walml/beta/galaxy_zoo/gz2' , '/home/walml/repos/galaxy-datasets/roots/gz2')
/tmp/ipykernel_24973/3819007290.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subfolder'] = df['file_loc'].apply(lambda x: os.path.basename(os.path.dirname(x)))


In [25]:
# # now check all the jpgs made correctly
df['file_exists'] = df['file_loc'].apply(os.path.isfile)
df['file_exists'].value_counts()


/tmp/ipykernel_24973/2637931321.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['file_exists'] = df['file_loc'].apply(os.path.isfile)


True    209293
Name: file_exists, dtype: int64

In [26]:
from PIL import Image

def is_formatted_well(loc, careful=False):
    try:
        im =  Image.open(loc)
        if careful:
            mean = np.array(im).mean()  # avoid lazy loading (very slow now, though, )
        return im.size == (424, 424)
    except:
        return False

# check all jpgs open correctly - if not, delete the bad ones and remake
df['is_formatted_well'] = df['file_loc'].apply(is_formatted_well)
df['is_formatted_well'].value_counts()


/tmp/ipykernel_24973/2714456404.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_formatted_well'] = df['file_loc'].apply(is_formatted_well)


True    209293
Name: is_formatted_well, dtype: int64

In [16]:
# now ready to be tarred and uploaded to dropbox

In [17]:
# TODO cross-match on sky with NSA here, for iauname

# first need to join to hard to get ra/dec
nsa = pd.read_parquet('/run/media/walml/beta/galaxy_zoo/decals/catalogs/nsa_v1_0_1_mag_cols.parquet', columns=['ra', 'dec', 'iauname'])

In [18]:


matched, unmatched = match_galaxies_to_catalog_pandas(df, nsa)
len(matched), len(unmatched)



/nvme1/scratch/walml/repos/shared-astro-utilities/shared_astro_utils/matching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  galaxies['best_match'] = best_match_catalog_index
/nvme1/scratch/walml/repos/shared-astro-utilities/shared_astro_utils/matching_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  galaxies['sky_separation'] = sky_separation.to(units.arcsec).value


(191768, 17525)

In [19]:
df = pd.merge(df, matched[['iauname', 'dr7objid']], on='dr7objid', how='left')  # some iaunames will be nan

In [20]:

df.to_parquet('/run/media/walml/beta/galaxy_zoo/gz2/subjects/gz2_catalog_with_modern_schema.parquet', index=False)